<a href="https://colab.research.google.com/github/paraery/parsingResume/blob/main/spacyDistillBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces only if the space does not occur between square brackets
    # we do not want to split "multi-word" entity value yet
    
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []
    lstEnt=[]
    txt = raw_text
    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")
            
            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            #txt = raw_text[raw_text.find(str(raw_entity_value))-1 : ]
            txt = raw_text.replace(raw_text[int(raw_text.find(raw_entity_value))-1],'',1)
            txt = txt.replace('](PERSON)','')
            #print(txt)
            lstdata = ()
            entity_name = f"{raw_entity_name}"
            entName = createSubEntity(txt.find(raw_entity_value), txt.find(raw_entity_value) + len(raw_entity_value), entity_name)
            lstEnt.append(entName)
                #tokens_with_entities.append((raw_entity_token, entity_name))
    d={"entities": lstEnt}
            #print(lstEnt)
    lstdata=(txt,d)
    tokens_with_entities.append(lstdata)
    return lstdata
def createSubEntity(start,end,tag):
    s = (int(start),int(end),tag)
    return s

In [17]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textfortraining8.csv')
lstTxt = df['text'].values.tolist()

In [18]:
#train_data=[]
for a in lstTxt:
    train_data.append(get_tokens_with_entities(a))

In [19]:
train=[]
train=train_data

In [20]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/testData.csv')
lstTest = df['text'].values.tolist()

In [21]:
test=[]
for a in lstTest:
    test.append(get_tokens_with_entities(a))

In [22]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [23]:
!pip install spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [24]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [25]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [26]:
!python -m spacy init fill-config /content/drive/MyDrive/Spacy/base_config.cfg /content/drive/MyDrive/Spacy/config.cfg

2023-03-04 11:47:03.344387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:47:03.344480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:47:03.344497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Spacy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train co

In [27]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
      except:
        continue
      if span is None:
        err_data = str([start,end])+" "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
      
      doc.ents = ents
      db.add(doc)
  return db

In [28]:
file = open('drive/MyDrive/Spacy/error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 87/87 [00:00<00:00, 164.68it/s]


In [29]:
!python -m spacy train /content/drive/MyDrive/Spacy/config.cfg --output ./outputSpacyDistillBert --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-03-04 11:49:42.823756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:49:42.823859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:49:42.823878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Created output directory: outputSpacyDistillBert
ℹ Saving to output directory: outputSpacyDistillBert
ℹ Using GPU: 0

=========================== Initializing pipeli

In [30]:
nlp = spacy.load('outputSpacyDistillBert/model-best')

In [31]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textInResume.csv')
dataTest = df['text'].values.tolist()

In [70]:
import re
i=0

bst_pred=[]
for d in dataTest:
  doc = nlp(d)
  print("## doc ",i," ##")
  person = ''
  for ent in doc.ents:
    nameRegex = re.compile('[A-Za-z]{2,25} [A-Za-z]{2,25}')
    if nameRegex.findall(ent.text)!=[]:
      print(ent.text)
      person=ent.text
  bst_pred.append(person)

    #print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
supavit attagomol
   
## doc  1  ##
nisaratwisesbantao bachelor
   
## doc  2  ##
thirawit jirarungroj
   
## doc  3  ##
   
## doc  4  ##
thanchanok watcharakitphokin
   
## doc  5  ##
   
## doc  6  ##
patcharaya anuntasinkul
   
## doc  7  ##
jarrukorn pensalaphan
   
## doc  8  ##
pravittra vimonworachort
   
## doc  9  ##
nattacha kingbualuang
   
## doc  10  ##
purawat ruangsri
   
## doc  11  ##
nutratanon mahakhet
   
## doc  12  ##
kitisak thossaensin
   
## doc  13  ##
suebtas limsaihua
   
## doc  14  ##
navara sirijarusvong
   
## doc  15  ##
chayanit sripradit
   
## doc  16  ##
pimnares puto
   
## doc  17  ##
saran thitawiriyayos
   
## doc  18  ##
pakpoom poodsud
   
## doc  19  ##
kanittha setthapitayakul
   
## doc  20  ##
   
## doc  21  ##
pawith panyasirikul
   
## doc  22  ##
chotitouch supalanunt
   
## doc  23  ##
aussadach masun
   
## doc  24  ##
teeratach jearapaganon
   
## doc  25  ##
pratan srikamonpattanawut
   
## doc  26  ##
natthapat phus

In [75]:
lstTrue

['Supavit Attagomol',
 'Nisarat Wisesbantao',
 'worakron jirarungroj',
 'Worawut Neammai',
 'Thanchanok Watcharakitphokin',
 'Natanop Pimonsathian',
 'patcharaya anuntasinkul',
 'jarrukorn pensalaphan',
 'pravittra vimonworachort',
 'Nattacha Kingbualuang',
 'purawat ruangsri',
 'nutratanon mahakhet',
 'kitisak thossaensin',
 'suebtas limsaihua',
 'navara sirijarusvong',
 'Chayanit Sripradit',
 'pimnares puto',
 'saran thitawiriyayos',
 'pakpoom poodsud',
 'kanittha setthapitayakul',
 'parweenuch duangdenphatsawron',
 'pawith panyasirikul',
 'chotitouch supalanunt',
 'aussadach masun',
 'teeratach jearapaganon',
 'pratan srikamonpattanawut',
 'Natthapat Phusrisom',
 'chayanon juntarapartsavorn',
 'narurong saeheng',
 'supachai sumeteenarumit',
 'ittichote sornmeethong',
 'kamolchai suebnipon',
 'surasak intasorn',
 'nattachai noogure',
 'Saksorn Pawasakarin',
 'navapuvadol uraikul',
 'Harit Piyapornthana',
 'setthawuth kangwansakol',
 'ratchanok angkhahad',
 'veerachaimitmorn',
 'Phanc

In [71]:
bst_pred

['supavit attagomol',
 'nisaratwisesbantao bachelor',
 'thirawit jirarungroj',
 '',
 'thanchanok watcharakitphokin',
 '',
 'patcharaya anuntasinkul',
 'jarrukorn pensalaphan',
 'pravittra vimonworachort',
 'nattacha kingbualuang',
 'purawat ruangsri',
 'nutratanon mahakhet',
 'kitisak thossaensin',
 'suebtas limsaihua',
 'navara sirijarusvong',
 'chayanit sripradit',
 'pimnares puto',
 'saran thitawiriyayos',
 'pakpoom poodsud',
 'kanittha setthapitayakul',
 '',
 'pawith panyasirikul',
 'chotitouch supalanunt',
 'aussadach masun',
 'teeratach jearapaganon',
 'pratan srikamonpattanawut',
 'natthapat phusrisom',
 'chayanon juntarapartsavorn',
 'narurong saeheng',
 'supachai sumeteenarumit',
 'ittichote sornmeethong',
 'kamolchai suebnipon',
 '',
 'nattachai noogure',
 'saksorn pawasakarin',
 'navapuvadol uraikul',
 'harit piyapornthana',
 'setthawuth kangwansakol',
 '',
 'veerachaimitmorn creativity',
 'phanchita korsanankittipat',
 'suphakit sangthong',
 '',
 'jitawat chanpraneet',
 '',

In [73]:
lst_pred

['supavit attagomol',
 'nisaratwisesbantao bachelor',
 'thirawit jirarungroj',
 '',
 'thanchanok watcharakitphokin',
 '',
 'patcharaya anuntasinkul',
 'jarrukorn pensalaphan',
 'pravittra vimonworachort',
 'nattacha kingbualuang',
 'purawat ruangsri',
 'nutratanon mahakhet',
 'kitisak thossaensin',
 'suebtas limsaihua',
 'navara sirijarusvong',
 'chayanit sripradit',
 'pimnares puto',
 'saran thitawiriyayos',
 'pakpoom poodsud',
 'kanittha setthapitayakul',
 '',
 'pawith panyasirikul',
 'chotitouch supalanunt',
 'aussadach masun',
 'teeratach jearapaganon',
 'pratan srikamonpattanawut',
 'natthapat phusrisom',
 'chayanon juntarapartsavorn',
 'narurong saeheng',
 '',
 'ittichote sornmeethong',
 'kamolchai suebnipon',
 '',
 'nattachai noogure',
 'saksorn pawasakarin',
 'navapuvadol uraikul',
 'harit piyapornthana',
 'setthawuth kangwansakol',
 '',
 'veerachaimitmorn creativity',
 'phanchita korsanankittipat',
 'suphakit sangthong',
 '',
 'jitawat chanpraneet',
 '',
 'suwan panjanapongcha

In [68]:
nlpLst = spacy.load('outputSpacyDistillBert/model-last')

In [72]:
import re
i=0
lst_pred=[]
for d in dataTest:
  doc = nlpLst(d)
  print("## doc ",i," ##")
  person = ''
  for ent in doc.ents:
    nameRegex = re.compile('[A-Za-z]{2,25} [A-Za-z]{2,25}')
    if nameRegex.findall(ent.text)!=[]:
      print(ent.text)
      person=ent.text
  lst_pred.append(person)

    #print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
supavit attagomol
   
## doc  1  ##
nisaratwisesbantao bachelor
   
## doc  2  ##
thirawit jirarungroj
   
## doc  3  ##
   
## doc  4  ##
thanchanok watcharakitphokin
   
## doc  5  ##
   
## doc  6  ##
patcharaya anuntasinkul
   
## doc  7  ##
jarrukorn pensalaphan
   
## doc  8  ##
pravittra vimonworachort
   
## doc  9  ##
nattacha kingbualuang
   
## doc  10  ##
purawat ruangsri
   
## doc  11  ##
nutratanon mahakhet
   
## doc  12  ##
kitisak thossaensin
   
## doc  13  ##
suebtas limsaihua
   
## doc  14  ##
navara sirijarusvong
   
## doc  15  ##
chayanit sripradit
   
## doc  16  ##
pimnares puto
   
## doc  17  ##
saran thitawiriyayos
   
## doc  18  ##
pakpoom poodsud
   
## doc  19  ##
kanittha setthapitayakul
   
## doc  20  ##
   
## doc  21  ##
pawith panyasirikul
   
## doc  22  ##
chotitouch supalanunt
   
## doc  23  ##
aussadach masun
   
## doc  24  ##
teeratach jearapaganon
   
## doc  25  ##
pratan srikamonpattanawut
   
## doc  26  ##
natthapat phus

In [74]:
import pandas as pd
dfName = pd.read_csv('drive/MyDrive/resumeData.csv')
lstTrue = dfName['PERSON'].values.tolist()

#Metrix of model best

In [81]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


In [92]:
y_true = np.array(lstTrue)
y_pred = np.array(bst_pred)

In [93]:
precision_recall_fscore_support(y_true, y_pred, average='weighted')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6609195402298851, 0.6666666666666666, 0.6628352490421456, None)

In [86]:
precision_recall_fscore_support(y_true, y_pred, average='macro')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.575, 0.58, 0.5766666666666667, None)

In [87]:
precision_recall_fscore_support(y_true, y_pred, average='micro')

(0.6666666666666666, 0.6666666666666666, 0.6666666666666666, None)

In [90]:
precision_recall_fscore_support(y_true, y_pred, average=None)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. ,
        1. , 0. , 1. , 0. , 0. , 1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ,
        0. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 0. , 1. , 1. ,
        0. , 1. , 0. , 0. , 0. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 0. ,
        1. , 0. , 1. , 1. , 1. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ,
        1. , 0. , 0. , 1. , 0. , 0. , 1. , 0. , 1. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.

#Metrix of model last

In [94]:
y_true = np.array(lstTrue)
y_pred = np.array(lst_pred)

In [95]:
precision_recall_fscore_support(y_true, y_pred, average='weighted')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6494252873563219, 0.6551724137931034, 0.6513409961685824, None)

In [96]:
precision_recall_fscore_support(y_true, y_pred, average='macro')

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.565, 0.57, 0.5666666666666668, None)

In [97]:
precision_recall_fscore_support(y_true, y_pred, average='micro')

(0.6551724137931034, 0.6551724137931034, 0.6551724137931034, None)

In [98]:
precision_recall_fscore_support(y_true, y_pred, average=None)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. ,
        1. , 0. , 1. , 0. , 0. , 1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ,
        0. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 0. , 1. , 1. ,
        0. , 1. , 0. , 0. , 0. , 0.5, 1. , 1. , 1. , 1. , 1. , 0. , 0. ,
        1. , 0. , 1. , 1. , 1. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ,
        1. , 0. , 0. , 1. , 0. , 0. , 1. , 0. , 1. ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.